In [31]:
import numpy as np
from scipy import spatial,stats
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
window_items =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17, 84, 121, 205, 206, 207, 211, 212, 213, 214, 215, 216, 217, 219, 220, 233, 234, 236, 239, 241, 245, 246, 247, 250, 251, 253, 254, 255]
target_items = [210, 235, 243, 244, 248, 249, 252, 256, 257] 

In [33]:
import os,sys
my_path = os.getcwd()
print (my_path)
path = os.path.join(my_path, "../data/ratings.txt")
print (path)

/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation
/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/Item-Based-Recommendation/../data/ratings.txt


In [34]:
train = pd.read_csv(path,sep=' ',header=None,names=['userId','itemId','rating'])

In [35]:
train.describe()

,userId,itemId,rating
count,35497.000000,35497.000000,35497.000000
mean,744.247824,265.524946,3.002803
std,433.199457,346.854769,0.918692
min,1.000000,1.000000,0.500000
25%,356.000000,12.000000,2.500000
50%,735.000000,216.000000,3.000000
75%,1130.000000,251.000000,4.000000
max,1508.000000,2071.000000,4.000000


### Actual Average Rating of each item 

In [36]:
calculated_Rating = {}
items = train['itemId'].unique()

#rating
R = train.groupby(['itemId'])['rating'].mean()
for i in items:
#     print (i)
    calculated_Rating[i] = R[i]
    
# print (calculated_Rating)

In [37]:
test = pd.DataFrame()
for t_i in target_items:
    temp_df = train[train['itemId'] == t_i]
    train =  train.drop(train[train['itemId'] == t_i].index)
    test = pd.concat([temp_df, test]).drop_duplicates(keep=False)

In [38]:
test.shape

(3539, 3)

### testing code of divising test/target data
table = [['Heading1', 'Heading2'], [1 , 1],[1 , 2],[1 , 3],[1 , 4], [1,5],[2 , 1],[2 , 2],[2 , 3],[2 , 4],[3, 1],[3, 2],[3, 3],[3, 4]]  
headers = table.pop(0)  
df = pd.DataFrame(table, columns=headers)

In [39]:
dfs = {n: g for n, g in test.groupby(test.groupby('itemId').cumcount() % 10)} 
test = pd.concat([dfs[0],dfs[1]], ignore_index=False)
train = pd.concat([train,dfs[2],dfs[3],dfs[4]], ignore_index=False)
del_data = pd.concat([dfs[6],dfs[7],dfs[8],dfs[9]], ignore_index=False)

In [40]:
train.shape[0]

33027

In [41]:
test.shape[0] + train.shape[0] + del_data.shape[0]

35144

In [42]:
test.tail(5)

,userId,itemId,rating
31088,1304,210,1.0
32543,1386,210,4.0
33313,1428,210,3.0
34102,1458,210,4.0
34776,1478,210,2.5


In [43]:
train.tail(5)

,userId,itemId,rating
31647,1338,210,2.0
32701,1396,210,4.0
33617,1439,210,2.0
34287,1462,210,2.0
34946,1486,210,3.5


In [44]:
del_data.tail(5)

,userId,itemId,rating
31019,1300,210,2.0
32273,1375,210,3.5
33186,1423,210,4.0
33990,1454,210,2.0
34655,1475,210,2.5


### Finding error for target items in actual data

In [45]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{210: [0, 0], 235: [0, 0], 243: [0, 0], 244: [0, 0], 248: [0, 0], 249: [0, 0], 252: [0, 0], 256: [0, 0], 257: [0, 0]}


In [46]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(list_of_movies_current_user_has_rated.itertuples(),1): 
        c1=[]
        c2=[]
        #c_user=int(u['userId'])
        c_item=int(u.itemId)
        #list_curuser=train[train['userId']==c_user]
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( all_ratings_for_the_active_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=stats.pearsonr(c1_list,c2_list)[0]#(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

/home/surya/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [47]:
index

714

In [48]:
print (t2-t1)

23.69926404953003


In [49]:
target_item_error_in_actual_data

{210: [nan, 78],
 235: [nan, 120],
 243: [nan, 66],
 244: [nan, 74],
 248: [nan, 60],
 249: [nan, 68],
 252: [nan, 76],
 256: [nan, 66],
 257: [nan, 106]}

### MAE error

In [50]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

210     : nan
235     : nan
243     : nan
244     : nan
248     : nan
249     : nan
252     : nan
256     : nan
257     : nan


## Push Attack Strength Check

### Injecting Attack Profiles

In [51]:
push_rating = 4

In [52]:
attack_ids = list(range(1600,1700))

for attacker in attack_ids:
    for item in window_items:
        a = calculated_Rating[item]
        attack_rating = int(a) if (a-int(a)<0.5) else int(a)+1
        train = train.append(pd.Series([attacker,item,attack_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

for attacker in attack_ids:
    for item in target_items:
        train = train.append(pd.Series([attacker,item,push_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

In [53]:
train.shape[0]

38027

In [54]:
train.tail(5)

,userId,itemId,rating
38022,1699,248,4.0
38023,1699,249,4.0
38024,1699,252,4.0
38025,1699,256,4.0
38026,1699,257,4.0


## Finding error for target items in attack profile injected data

In [55]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{210: [0, 0], 235: [0, 0], 243: [0, 0], 244: [0, 0], 248: [0, 0], 249: [0, 0], 252: [0, 0], 256: [0, 0], 257: [0, 0]}


In [56]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(list_of_movies_current_user_has_rated.itertuples(),1): 
        c1=[]
        c2=[]
        #c_user=int(u['userId'])
        c_item=int(u.itemId)
        #list_curuser=train[train['userId']==c_user]
        list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( all_ratings_for_the_active_item,list_curitem,how='inner',on=['userId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim=stats.pearsonr(c1_list,c2_list)[0]#(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

/home/surya/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [57]:
print (t2-t1)

27.694863319396973


In [58]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

210     : nan
235     : nan
243     : nan
244     : nan
248     : nan
249     : nan
252     : nan
256     : nan
257     : nan


In [ ]:
210     : 2.3058171134229086
235     : 2.1928669466604225
243     : 2.1907303256937363
244     : 2.349442613055623
248     : 2.4582805479544607
249     : 2.396745080071849
252     : 2.286436759922374
256     : 2.1500551206142404
257     : 2.4654589180088586
